# Multidimensional genetic programming for multiclass classification

This notebook is based on:  
__W. La Cava, S. Silva, K. Danai, L. Spector, L. Vanneschi, J.H. Moore,   
Multidimensional genetic programming for multiclass classification,   
Swarm and Evolutionary Computation BASE DATA (2018),  
doi: 10.1016/j.swevo.2018.03.015.__   
Article available at: https://www.sciencedirect.com/science/article/abs/pii/S2210650217309136?via%3Dihub  

Some explanation parts of this notebook may directly quote or paraphrase this article or other sources for clarity purposes.

The included code has been developed for the ISAE-Supaero SDD evaluation in order to showcase how genetic programming can be used for feature engineering. 

__Author__: Student 57 (anonimized)  
__License__: CC-BY-SA-NC.

## Requirements 
The following libraries for Python 3.6+ are required:
- numpy
- matplotlib
- pandas
- sklearn
- seaborn 

If they are not installed, run:   
`!pip install numpy matplotlib seaborn pandas sklearn`   
`!pip update numpy matplotlib seaborn pandas sklearn`

The code for this notebook is stored in `./Code` and loaded when needed.  
When you encounter a cell with `%load Code/name.py`, run it twice: once to load the code, once to execute it.  
If the `%load` command is commented, this is meant to be a solution to the question. Uncomment it and run it twice as well to load the code.

In [ ]:
# Imports
import numpy as np
import pandas as pd
import random as rd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestCentroid
from sklearn.base import clone
import seaborn as sns

<div class="alert alert-success"><b>In a nutshell:</b>
<ul>
<li> Feature Selection and Feature Construction can create a new feature space where the samples are more accurately classified by a Nearest Centroid model
<li> Genetic Programming evolves transformations of the feature space to select the ones giving the best classification results 
<li> Evaluating intelligibility as well as performance in the fitness calculation allows to maintain good performance while avoiding black-box models like neural networks
</ul>
</div>

## Introduction

### Feature Selection and Intelligibility

Feature selection and feature construction play fundamental roles in the application of machine learning (ML) to classiﬁcation. Feature selection makes it possible, for example, to reduce high-dimensional datasets to a manageable size, and to reﬁne experimental designs through measurement selection in some domains. The ML community has become increasingly aware of the need for automated and ﬂexible feature engineering methods to complement the large set of classiﬁcation methodologies that are now widely available in opensource packages such as Weka and Scikit-Learn.  

Typical classiﬁcation pipelines treat feature selection and feature construction as pre-processing steps, in which the attributes in the dataset are selected according to some heuristic and then projected into more complex feature spaces using e.g. kernel functions. In both cases the feature pre-processing is often conducted in a trial-and-error way rather than being automated or intrinsic to the learning method. The use of non-linear feature expansions can also lead to classiﬁers that are black-box, making it diﬃcult for researchers to gain insight into the modelled process by studying the model itself. 

This paper investigates a multiclass classiﬁcation strategy designed to integrate feature selection, construction and model intelligibility goals into a distance-based classiﬁer to improve its ability to build accurate and simple classiﬁers.


## Datasets

We introduce 2 datasets for multiclass classification.  
Each is imported and converted into a pandas.DataFrame, and a 10% validation set is kept for the final selection. 

### Iris

The first toy dataset we can use for multiclass classification is the very well known `Iris dataset`.

In [ ]:
iris_load = datasets.load_iris()
iris = pd.DataFrame(iris_load['data'], columns=['sep_l', 'sep_w', 'pet_l', 'pet_w'])
iris['target']=iris_load['target']
iris, iris_val = train_test_split(iris, test_size=0.1)
iris.head()

### Protein Localization Sites

From: https://archive.ics.uci.edu/ml/datasets/Yeast  
Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

This dataset, also called `Yeast dataset`, has been created in 1996. Its goal is to use statistical modeling to predict the cellular localization of proteins from their attributes.

__Results__: __55%__ for Yeast data with an ad hoc structured probability model. 
Also similar accuracy for Binary Decision Tree and Bayesian Classifier methods applied by the same authors in unpublished results.


Features in this dataset:
-  Sequence Name: Accession number for the SWISS-PROT database
-   mcg: McGeoch's method for signal sequence recognition.
-   gvh: von Heijne's method for signal sequence recognition.
-   alm: Score of the ALOM membrane spanning region prediction program.
-   mit: Score of discriminant analysis of the amino acid content of the N-terminal region (20 residues long) of mitochondrial and non-mitochondrial proteins.
-   erl: Presence of "HDEL" substring (thought to act as a signal for retention in the endoplasmic reticulum lumen). Binary attribute.
-   pox: Peroxisomal targeting signal in the C-terminus.
-   vac: Score of discriminant analysis of the amino acid content of vacuolar and extracellular proteins.
-   nuc: Score of discriminant analysis of nuclear localization signals of nuclear and non-nuclear proteins.
  
The *Sequence Name* attribute is dropped as it is unique to each data point.
The predicted class is encoded in the *target* column to fit the rest of the code.

For more information, run `%load Data/yeast.names` in a cell.

In [ ]:
yeast = pd.read_csv('Data/yeast.data', sep='\s+', header=None)
yeast.columns = ['Sequence_name', 'MCG', 'GVH', 'ALM', 'MIT', 'ERL', 'POX', 'VAC', 'NUC', 'target']
yeast['target'] = yeast['target'].astype('category').cat.codes
yeast = yeast.drop(columns=['Sequence_name', 'POX'])
yeast, yeast_val = train_test_split(yeast, test_size=0.1)
yeast.head()

In [47]:
# Switch between yeast and iris to choose the one you want to use
data = yeast 

if data is yeast:
    data_val = yeast_val
elif data is iris:
    data_val = iris_val
    
data.shape


(1335, 8)

<div class="alert alert-success"><b>For later use</b><br>
    You can use this notebook with your own data to automate feature engineering for your classification projects! <br>
    Import your dataset as <code>data</code>, with a <code>target</code> column for the classes to predict, and your validation set as <code>data_val</code>.<br>
    Or indicate the path to you csv in the next cell.<br><br>
    Then, run the notebook!
</div>

(You may have to pre-process your dataset to avoid unforseen bugs and/or conflicts with the models)

In [ ]:
path = #TODO
df = pd.read_csv(path)
data, data_val = train_test_split(df, test_size=0.1)

# Checking the data format 
assert 'target' in data.columns
for c in data.columns:
    assert type(data[c]) is not str

data.head()

### Data Exploration

<div class="alert alert-warning"><b>Code</b><br>
    Explore the <code>yeast</code> dataset. <br>
    Do you see any class easily separable from the rest?
</div>

In [ ]:
i, j = 0, 1
col = data.columns
fig, ax = plt.subplots(figsize=(8, 8))
scatter = ax.scatter(data[col[i]], data[col[j]], c=data['target'])
plt.xlabel(col[i])
plt.ylabel(col[j])
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="lower right", title="Classes")
ax.add_artist(legend1)
plt.show()

### Multiclass classification

The goal of multiclass classiﬁcation is to ﬁnd a mapping between any vector $x \in \mathbb{R}^p $ and a class label from the set $C = \{1 ... K\}$, with $K > 2$:

$$ \hat{y}(x) : \mathbb{R}^p \to C $$

using a training set $T = \{(x_i,y_i),i = 1 ... n\}$ that associates each example $x_i \in \mathbb{R}^p $ to a label $y_i \in C$.

The *Nearest Centroid* algorithm conducts classiﬁcation by measuring the similarity of each vector to the bulk properties of the examples within each class, and then assigning the label corresponding to the most similar group.    
The attributes are partitioned into subsets $\{X_1 ... X_K\}$, such that $X_k$ is the subset of x with class label k:
$$X_k = \{x_i | y_i == k\}$$


To classify a new sample $x' \in \mathbb{R}^p$, the distance of $x'$ to each subset $\{X_1 ... X_K\}$ is measured and the class label corresponding to the minimum distance is assigned to $x'$:  
$$ \hat{y}(x') = j, \:where\: j= \underset{k}{argmin}\: D(x', X_k) $$

$D$ being a distance operator between $x'$ and the centroid of $X_k$.  
Previous work indicates the [Mahalanobis distance](https://en.wikipedia.org/wiki/Mahalanobis_distance) to perform better on classification than the Euclidean distance.  

We compute the mean accuracy on the given test data and labels.   
In multi-label classification, this is the subset accuracy, indicating the percentage of samples that have all their labels classified correctly.

__Note:__
Since this notebook will rely on the `sklearn` library to perform Nearest Centroid classification, the sensitivity of the matrix inversion (used in the computation of the Mahalanobis distance) in the `numpy` library to any linear combination of features will lead us to use Euclidean distance in the implementation. 

In [ ]:
def compute_score(data, target='target'):
    df = data.drop(columns=['target'])
    model = NearestCentroid(metric='euclidean')
    X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    data[target], 
                                                    test_size=0.3, 
                                                    random_state=42)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    return score

In [ ]:
s = compute_score(data, target='target')
print("Score:", s)

## Feature transformation

### Limits of Nearest Centroid classification

<img src="img/example.png" width="800">
<center><a href='#Sources'>Source: Multidimensional genetic programming for multiclass classification</a></center>

The nearest centroid classiﬁer assumes the within-class samples to be normally distributed about their centroid and separated from the distributions of other classes. For this reason, the data in the right plot is easily classiﬁable using nearest centroids. Unfortunately, many real-world problems have data that violates these assumptions, as in the left plot.  
Furthermore, for high dimensional data, calculating the distance between a point and a centroid can be impractical. However, by ﬁnding a set of transformations that project the data into a space that delineates class membership, the performance of a nearest centroid classiﬁer can be improved and high-dimensional distance comparisons may be avoided. Furthermore, the relationship between the raw data and the transformed space can be made understandable by using symbolic transformations, as in $\phi_1$ and $\phi_2$. 

The goal of the algorithm described in this paper is to ﬁnd a set of transformations 
$\phi(x) : \mathbb{R}^p \to \mathbb{R}^d $
that projects $x$ into a space in which the samples are more accurately classiﬁed by the nearest centroid method. 

Classiﬁcation is then conducted with centroids $\mu_{\phi_k} \in \mathbb{R}^d$ and distances $D(\phi(x'), \phi(X_k))$.
Our goal will be to approximate the optimal synthesized features $\phi^* = [\phi_1 ... \phi_d]$ that maximize the number of correctly classiﬁed training samples, as:

$$ \phi^*(x) = \underset{\phi \in \mathbb{S}}{argmax}\: f(\phi, \mathcal{T})$$

$$ f(\phi, \mathcal{T}) = \frac{1}{n}\sum_{i=1}^{n} \mathbb{I}(\hat{y}(\phi(x_i))=y_i)$$ 

where $\mathbb{S}$ is the space of possible transformations $\phi$, $f$ is the classiﬁcation accuracy, and the indicator function  $\mathbb{I} = 1$ if $\hat{y}(\phi(x_i))=y_i$, and 0 otherwise. 

### *Implementation*: `Var` class

In order to apply operations to features more easily, we define a `Var` object.
A `Var` object represents a feature or combination of features through a basic operation.
Hence, it can be of 3 types:
- feature from the dataset
- 1-parameter function of another `Var` object: $sin, cos, exp$
- 2-parameters function of 2 other `Var` objects: $+, -, \times, \div $

The last 2 categories are summed up in the `Var.FUNCTIONS` dictionaries linking function names to functions.
- `FUNCTIONS[1]` lists 1-parameter functions
- `FUNCTIONS[2]` lists 2-parameters functions

If an element is in neither of them, it is considered as a feature from the dataset.

__Run the following cell to load the code, then run it again to use it.__

In [ ]:
%load Code/var.py

Since $sin, cos$ and $exp$ are defined in the `math` library, we extend their range thanks to a wrapper.
If one of these functions is called on a `Var` object, it will call the method from the `Var` class. On anything else, it will proceed as normal.

In [ ]:
import math
# Sin, Cos and Exp are redefined to be applicable on Var objects
def wrap_function(func):
    def wrapper(x):
        if type(x) is Var:
            if func.__name__ == 'sin':
                result = x.sin()
            elif func.__name__ == 'cos':
                result = x.cos()
            elif func.__name__ == 'exp':
                result = x.exp()
            elif func.__name__ == 'log':
                result = x.log()
        else: 
            result = func(x)
        return result
    return wrapper

sin = wrap_function(math.sin)
cos = wrap_function(math.cos)
exp = wrap_function(math.exp)
log = wrap_function(math.log)

Any basic operation can be done on `Var`objects easily.

Ex:<code>  
length = Var(name='length')  
width  = Var(name='width')  
area   = length * width  
</code> 

`area` is a `Var` and can be used as such


`Var.compute(data)` computes the column of the new dataset corresponding to the operation defined by this `Var` from the initial dataset `data`.

<div class="alert alert-warning"><b>Code</b><br>
Create, compute and plot the following features for the <code>Iris</code> dataset with this class:
<ul>
<li> sepal length + petal length
<li> petal length * petal width
<li> sin(exp(petal length))
</ul>
</div>
Remember: the column names are: 'sep_l', 'sep_w', 'pet_l', 'pet_w'.

In [ ]:
# Uncomment to load the solution
#%load Code/sol_iris.py

## Genetic Programming

Genetic programming is an evolutionary computation technique where a population of computer programs or functions are evolved. Generation by generation, Genetic Programming stochastically transforms populations of programs into new, hopefully better, populations of programs.

It is a variant to Genetic Algorithms since it optimizes programs to find the optimum of the fitness function, designed as a measure of the efficiency of a solution to solve the problem at hand.   
As it is a stochastic (ie random) process results cannot be quaranteed, but it allows to escape traps in which deterministic methods may fall. 


### Program representation

Each program $\phi(x) : \mathbb{R}^p \to \mathbb{R}^d $ can be seen as a set of equations. For example, an individual program $i$ might encode the features:

$$ i \to \phi(x) = [x_1, x_2, x_1^2, x_2^2, x_1 x_2]$$


This work implements a stack-based representation of the equations that create the new features. Programs in this representation are encoded as post-ﬁx notation equations, e.g.:
$ i = [x_1, x_2, +] \to \phi = [x_1 + x_2]$  
The $i$ representation is flattened: $ \phi = [(x_1 + x_2)*x_3] \to i = [x_1, x_2, +, x_3, *] $  

Programs are evaluated via executions on a stack, such that the program $i$ above can be constructed as
$$i = [ x_1,  x_2,  x_1,  x_1,  ∗,  x_2,  x_2,  ∗,  x_1,  x_2,  ∗ ]$$

The execution of program $i$ is illustrated in this figure:

<img src="img/Multidimensional_transformation.png" width="800">
<center><a href='#Sources'>Source: Multidimensional genetic programming for multiclass classification</a></center>

Stack based evaluation proceeds left to right, pushing and pulling instructions to and from a single stack. Arguments such as $x_1$ are pushed to the stack, and operators such as $*$ pull arguments from the stack and push the result. At the end of a program’s execution, the entire stack represents the multi-dimensional transformation.

#### Note - Implementation
In this implementation and to prevent infinite or `NaN` values in the dataset, operations have been restrained to:
$$+, -, *, sin, cos$$

Other operators such as $\div, log$ and $exp$ have been implemented but were not enabled as they either cannot compute null or negative values, or their output can outrange the machine capacity.  

To enable them, modify the `Var.FUNCTIONS` dictionary (not recommended in this notebook).

### Generating instructions
We define 2 lists of operators in the FUNCTIONS dictionary.

`FUNCTIONS = {
        1:['sin', 'cos', 'exp'],
        2:['+', '-', '*', '/']
    }
`

`FUNCTIONS [1]` lists functions that require 1 argument.   
`FUNCTIONS [2]` lists functions that require 2 arguments.

With `a` and `b` as variable names, a program of size 7 may look like the following list:
`['a', 'a', '+', 'b', 'sin', 'a', '*']`  
which once executed gives the following features list:
$$[a + a, \:\sin(b) * a]$$

<div class="alert alert-warning"><b>Code</b><br>
Complete next cell to generate random lists of instructions of size n from any list of variable names passed in argument.<br> 
What are the constraints linked to the number of arguments a function needs? <br>
Execute your program manually to separate the features.
</div>

In [ ]:
def generate(size, variables):
    """
    Return a random list of instructions.

    :param: size Int
        Number of instructions in the list
    :param: list of String
        List of the names of the variables to use to generate the instructions
    """
    FUNCTIONS = {
        1:['sin', 'cos', 'exp'],
        2:['+', '-', '*', '/']
    }
    
    #TODO
    
    return l

In [ ]:
# Uncomment to load the solution
# %load Code/generate.py

In [ ]:
# Test
v =['a', 'b', 'c']

l = generate(size=10, variables=v)
print(l)

### Splitting features

In order to do mutation and crossover operations on the features, we need to split the list of instructions into sub-lists of 1 feature each.  
Thus, we define a `split_features` method to return a list of features, aggregating the instructions in each of the root where they belong.

__Run the following cell to load the code, then run it again to use it.__

In [ ]:
%load Code/split_features.py

<div class="alert alert-warning"><b>Code</b><br>
    Generate a random instructions list with the code from the previous part and split it with this new method. <br>
    Is the result what you expected? If not, edit your <code>generate()</code> method.
</div>

In [ ]:
# Test
l = generate(size=10, variables=v)
print(l)
split_features(l)

### Operations on trees

Each new feature can be seen as a tree: 

<img src="img/tree.png" width="200">
<center><a href='#Sources'>Source: A Field Guide to Genetic Programming</a></center>

This tree implements the $x + 3 * y$ function, and can be represented as a stack:
$$i=[x, 3, y, *, +]$$

Hence, any genetic operators widely used on trees in Genetic Programming can be used to define Mutation and Crossover on the instruction lists.

#### Mutation

<img src="img/tree_mutation.png" width="400">
<center><a href='#Sources'>Source: A Field Guide to Genetic Programming</a></center>

The mutation operator, with equal probability, chooses one of three actions: 
- it replaces a sub-tree with a randomly generated sub-tree
- it adds a new randomly generated sub-tree to the end of the program, thereby increasing the number of features by one
- it deletes a sub-tree corresponding to a “root”, thereby reducing the number of features by one


#### Crossover

<img src="img/tree_crossover.png" width="400">
<center><a href='#Sources'>Source: A Field Guide to Genetic Programming</a></center>

The crossover operator similarly chooses between one of two equally probable actions: 
- it performs standard sub-tree crossover of the parents, selecting non-“root” nodes
- it performs standard sub-tree crossover of “root” nodes.   

In this case, “roots” are those nodes in the program that produce a value in the ﬁnal stack, and can be identiﬁed from stack-based programs in linear time.  

2 children are often generated during crossover by switching parts of the parents' trees. They are thus complementary wrt. their parents, hence we only keep one of them in the new population.


### `Program` class

A `Program` object contains a list of instructions and all the related tools to the transformation of features on a dataset.

__Run the following cell to load the code, then run it again to use it.__

In [ ]:
%load Code/programs.py

<div class="alert alert-warning"><b>Code</b><br>
    In the next cell we generate random <code>Program</code> objects for the <code>yeast</code> dataset, then make them mutate and crossover. <br>
    What type of mutation happened? <br>
    What type of crossover happened? <br>
    Can you retrace which parent each feature came from in the children?<br>
</div>

In [ ]:
# Var definition for each column except 'target'
v = {}
for i in data.columns[:-1]:
    v[i] = Var(name=i)
print("Variables:", list(v.keys()), '\n')

# Mutation
a = Program(v).generate(20).get_features()
print('>> A\n', a)

a.mutate()
print('>> A after mutation\n', a)

# Crossover: parents
p1 = Program(v).generate(10).get_features()
print('>> P1\n', p1)

p2 = Program(v).generate(10).get_features()
print('>> P2\n', p2)

# Children
c1, c2 = p1.crossover(p2)
print('>> Children:\n', c1, '\n', c2)


## Genetic Algorithm Structure

### Population creation and Generation cycles

The population of the Genetic Algorithm (GA) is initialized as an array of `Program` object, each randomly generated.  
At each generation, the population is evaluated and 3 genetic operators are applied to form the new population:
- Mutation: individuals are selected, mutated, and added to the new population
- Crossover: pairs of individuals are selected from the population, and their features are merged to create 2 children, between which the parents' features are divided. One child is added to the new population
- Reproduction: if the crossover and mutation operations do not create a list with the same length as the previous population, the best individuals are copied into the new population.

### Tournament selection

Tournament selection is a selection operator commonly used in Genetic Algorithms to select which individuals will be used in each of the genetic operators described above. 

Every time a new element needs to be picked - either to be mutated or to be used as a parent in a crossover -:
- $\lambda$ individuals are selected at random
- among them, the one with the highest fitness is chosen.

Here, the tournament only decides between 2 individuals ($\lambda = 2$). A higher $\lambda$ can be chosen for higer selection pressure.

### `Population` class

In [ ]:
%load Code/population.py

<div class="alert alert-warning"><b>Code</b><br>
    Test the <code>Population</code> class on the <code>Yeast</code> dataset and look through `p.archive`. <br>
    Is the maximum fitness improving? <br>
    Which solution would you recommend?
</div>

In [ ]:
# Evaluation
p = Population(data = yeast, init_size=15, pop_size=100)
p.generation(n=20, plot=True)

### Final Selection

#### Reminder: Pareto Dominance
When trying to minimize $f_1$ and $f_2$, $x_1$ dominates $x_2$ if:

$$\forall i \in \{1, 2\}, f_i(x_1) \leq f_i(x_2) $$
<center>and</center> $$\exists j \in \{1, 2\}, f_i(x_1) < f_i(x_2)$$

The Pareto front is defined as the set of non-dominated points:

<img src="img/pareto.png" width="400">

#### Archiving and final selection

Typically in GP the individual with the highest training accuracy (the so-called ‘best-of-run’ individual) is chosen as the ﬁnal model and evaluated on the test set.   
However, given the importance of interpretability, on subsequent problems we maintain an archive of solutions that represent the best trade-oﬀs of training accuracy and complexity (i.e. the Pareto set). To choose a ﬁnal model, a small validation set `data_val` is split from the training set before the run. At the end of the run, the archive is evaluated on the validation set, and the individual with the best score is chosen as the ﬁnal model. This approach should guard against over-ﬁtting and produce simpler models. The entire archive can also be studied by experts to get a better sense of the building blocks of good solutions to their problem, which can oﬀer insight to the user.

<div class="alert alert-warning"><b>Question</b><br>
    How did each solution in the archive adapt to the validation set? <br>
    Would you still recommend the same solution?
</div>

In [ ]:
p.final_selection(data_val)

## Other selection methods

### Lexicase Selection
In each parent selection event, the lexicase selection algorithm ﬁrst randomly orders the test cases. It then eliminates any individuals in the population that do not have the best performance on the ﬁrst test case. Assuming that more than one individual remains, it then loops, eliminating any individuals from the remaining candidates that do not have the best performance on the second test case. This process continues until only one individual remains and is selected, or until all test cases have been used, in which case it randomly selects one of the remaining individuals. 

Lexicase selection sometimes selects individuals that perform well on a relatively small number of test cases. This differs from most other selection algorithms, which select individuals based on aggregations of performance on all test cases. Lexicase selection may select individuals that perform very poorly on some test cases if they excel on a combination of others. As such, lexicase often selects “specialist” individuals that solve parts of the problem extremely well. Although these individuals may have worse summed error across all test cases, the hope is they will be able to reproduce in ways that pass on their preeminence on certain cases while improving with respect to others. In order to give every test case equal selection pressure, each lexicase selection event uses a randomly shufﬂed list of test cases to determine which test cases are treated as most important. 

#### Implementation

We use inheritance to define a new class `LexPopulation` from `Population` that will implement lexicase selection instead of tournament selection:

In [ ]:
%load Code/lexicase.py

Run next cell to perform the process with Lexicase Selection:

In [ ]:
p = LexPopulation(data = yeast, init_size=15, pop_size=100)
p.generation(n=20, plot=False)
p.final_selection(data_val)

### Age-Fitness Pareto survival

#### Non-Dominated Sorting

Non-Dominated Sorting (NDS) is an algorithm that computes a fitness from the number of other points dominated by each individual.  
NDS maps the solutions $x_i$ of a population $P$ to $K$ Pareto Fronts $F_1, ..., F_K$.   
It first selects all the non-dominated solutions from $P$ and assigns them to $F_1$ (rank 1). It then selects all the non-dominated solutions from the remaining population and assigns them to $F_2$ (the rank 2 Pareto front).   
This process is repeated untill all solutions have been assigned to a front.

<img src="img/nds.png" width="400">

The fitness is then computed from the number of individuals dominated by $x_i$ divided by the size of the global population:
$$F(x_i) = \frac{Number \: of\:solutions \: dominated \:by\:x_i}{Total\: number\: of\: solutions}$$
All solutions on the same Pareto front have the same fitness. 

#### Age-fitness Pareto survival

A common problem in many applications of evolutionary algorithms is when the progress of the algorithm stagnates and solutions stop improving. Expending additional computational effort in the evolution often fails to make any substantial progress. This problem is known as premature convergence.   
A common method for dealing with premature convergence is to perform many evolutionary searches, randomizing and restarting the search multiple times. This approach can be wasteful however, as the entire population is repeatedly thrown out. There is also the difficulty of deciding when to restart.  

Age-fitness Pareto survival is a multi-objective method using age as an independent dimension in a multi-objective Pareto front optimization to favor new solutions that might perform better.

The age of a solution is measured in generations. All randomly initialized individuals start with age of one. With each generation an individual exists in the population, its age is incremented by one. During crossover and mutation events, the age is inherited as the maximum age of the parents 

<img src="img/age_fitness_pareto.png" width="400">
<center><a href='#Sources'>Source: Age-Fitness Pareto Optimization</a></center>

#### Algorithm steps

First, the population is initialized randomly.  
At each generation, an extended population is created from:
- the previous population
- the mutation of randomly selected solutions
- the children through crossover of randomly selected parents
- new random solutions with age 0

We introduce the *Age-Fitness Pareto fitness*, called `AFP_fitness`, computed from the previous `fitness` (the performance of a solution on a test dataset) and the `age` through a Non-Dominated Sorting algorithm. 

This *Age-Fitness Pareto fitness* is used to select the best solutions in the extended population to create the next population.

#### Implementation

We use inheritance to define a new classes `AFP_Program` from `Program` to add age, and`AFPPopulation` from `Population` that will implement Age-Fitness Pareto survival instead of tournament selection:

In [ ]:
%load Code/afp_programs.py

In [ ]:
%load Code/afp_population.py

Run next cell to perform the process with Lexicase Selection:

In [ ]:
p = AFPPopulation(data = yeast, init_size=15, pop_size=100)
p.generation(10, plot=True)
p.final_selection(data_val)

## Results

The work this notebook is based on implements the M4GP algorithm, evolution of $M_2GP$ (Multi-dimensional Multi-class Genetic Programming).   
The best program generated by M4GP was compared to benchmark methods and other published results.

The presented selection methods are also compared:
- M4GP-tn: Tournament Selection
- M4GP-lx: Lexicase selection
- M4GP-ps: Age-fitness Pareto Survival (with a more advanced form than the algorithm presented in this notebook)

The `Yeast` benchmark is the `Yeast` dataset presented in this notebook.

<img src="img/benchmarks.png" width="800">
<center><a href='#Sources'>Source: Multidimensional genetic programming for multiclass classification</a></center>

M4GP represents an improvement over previous state-of-the-art techniques for multi-class classiﬁcation with GP (M2GP, M3GP and eM3GP).   
It also outperformed classical solutions, including XGBoost and Deep Neural Networks, on biomedical applications:


<img src="img/biomedical.png" width="800">
<center><a href='#Sources'>Source: Multidimensional genetic programming for multiclass classification</a></center>


## Sources 
<a id='Sources'>  
The pdf files of the sources used in this work are available in the `Source` repository.    
</a>  

Recommended readings:

1.__Multidimensional genetic programming for multiclass classification__  
La Cava, William & Silva, Sara & Danai, Kourosh & Spector, Lee & Vanneschi, Leonardo & Moore, Jason. (2018).     
Swarm and Evolutionary Computation. 10.1016/j.swevo.2018.03.015.    
   
2.__M4GP__   
La Cava, William & Silva, Sara & Vanneschi, Leonardo & Spector, Lee & Moore, Jason. (2017). Genetic Programming Representations for Multi-dimensional Feature Learning in Biomedical Classification. 158-173. 10.1007/978-3-319-55849-3_11. 
  
3.__A Field Guide to Genetic Programming__  
Riccardo Poli and William B. Langdon and Nicholas Freitag McPhee (2008)  
Published via http://lulu.com and freely available at http://www.gp-field-guide.org.uk    
  
4.__Age-Fitness Pareto Optimization__  
Schmidt, Michael & Lipson, Hod. (2010).   
Age-Fitness Pareto Optimization. 8. 543-544. 10.1145/1830483.1830584. 
Available [here](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.375.6168&rep=rep1&type=pdf)
  
5.__A Novel Non-Dominated Sorting Algorithm for Evolutionary Multi-objective Optimization__   
Bao, Chunteng & Xu, Lihong & Goodman, Erik & Cao, Leilei. (2017). A Novel Non-Dominated Sorting Algorithm for Evolutionary Multi-objective Optimization. Journal of Computational Science. 23. 10.1016/j.jocs.2017.09.015. 

